## Logistic regression

This notebook compares various logistic regression implementations. The dataset used is the [New South Wales electricity price dataset](https://creme-ml.github.io/generated/creme.datasets.fetch_electricity.html#creme.datasets.fetch_electricity).

In [1]:
%load_ext watermark
%watermark --python --machine --packages creme,keras,sklearn,tensorflow,torch --datename

Using TensorFlow backend.
/home/max/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/max/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/max/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/max/anaconda3/lib/python3.7/site-packages/tensorflow/python

Tue Oct 08 2019 

CPython 3.7.4
IPython 7.4.0

creme 0.3.0
keras 2.2.4
sklearn 0.21.3
tensorflow 1.14.0
torch 1.1.0

compiler   : GCC 7.3.0
system     : Linux
release    : 5.2.10-arch1-1-ARCH
machine    : x86_64
processor  : 
CPU cores  : 8
interpreter: 64bit


In [2]:
from creme import datasets
from creme import linear_model
from creme import metrics
from creme import optim
from creme import preprocessing
from keras import layers
from keras import models
from keras import optimizers
from sklearn import linear_model as sk_linear_model
import torch

%run utils.py
%run wrappers.py

In [3]:
n_features = 8
lr = 0.005

# PyTorch
class PyTorchNet(torch.nn.Module):
    
    def __init__(self, n_features):
        super().__init__()
        self.linear = torch.nn.Linear(n_features, 1)
        self.sigmoid = torch.nn.Sigmoid()
        torch.nn.init.constant_(self.linear.weight, 0)
        torch.nn.init.constant_(self.linear.bias, 0)
        
    def forward(self, x):
        return self.sigmoid(self.linear(x))
    
torch_model = PyTorchNet(n_features=n_features)

# Keras
inputs = layers.Input(shape=(n_features,))
predictions = layers.Dense(1, activation='sigmoid', kernel_initializer='zeros', bias_initializer='zeros')(inputs)
keras_model = models.Model(inputs=inputs, outputs=predictions)
keras_model.compile(optimizer=optimizers.SGD(lr=lr), loss='binary_crossentropy')


def add_hour(x):
    x['hour'] = x['moment'].hour
    return x

results = benchmark(
    get_X_y=datasets.fetch_electricity,
    n=45312,
    get_pp=preprocessing.StandardScaler,
    models=[
        ('creme', 'LogisticRegression', linear_model.LogisticRegression(
            optimizer=optim.SGD(lr),
            l2=0.,
            intercept_lr=lr
        )),

        ('scikit-learn', 'SGDClassifier', ScikitLearnClassifier(
            model=sk_linear_model.SGDClassifier(
                loss='log',
                learning_rate='constant',
                eta0=lr,
                penalty='none'
            ),
            classes=[False, True]
        )),
        
        ('PyTorch (CPU)', 'Linear', PyTorchBinaryClassifier(
            network=torch_model,
            loss_fn=torch.nn.BCELoss(),
            optimizer=torch.optim.SGD(torch_model.parameters(), lr=lr)
        )),
        
        ('Keras on Tensorflow (CPU)', 'Dense', KerasBinaryClassifier(
            model=keras_model
        )),
        
    ],
    get_metric=metrics.LogLoss
)

W1008 13:53:35.777601 139742666524480 deprecation_wrapper.py:119] From /home/max/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1008 13:53:35.793908 139742666524480 deprecation_wrapper.py:119] From /home/max/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1008 13:53:35.837010 139742666524480 deprecation_wrapper.py:119] From /home/max/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1008 13:53:35.844192 139742666524480 deprecation_wrapper.py:119] From /home/max/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W1008 13:53:35.851012 139742666524480

W1008 13:54:53.313901 139742666524480 deprecation_wrapper.py:119] From /home/max/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:2741: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W1008 13:54:53.317058 139742666524480 deprecation_wrapper.py:119] From /home/max/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



In [4]:
results

,Library,Model,LogLoss,Fit time,Average fit time,Predict time,Average predict time
0,creme,LogisticRegression,0.413533,"812ms, 676μs, 174ns","17μs, 935ns","245ms, 461μs, 635ns","5μs, 417ns"
1,scikit-learn,SGDClassifier,0.413533,"18s, 902ms, 205μs, 239ns","417μs, 157ns","6s, 652ms, 347μs, 9ns","146μs, 812ns"
2,PyTorch (CPU),Linear,0.418901,"30s, 274ms, 892μs, 26ns","668μs, 143ns","10s, 904ms, 302μs, 161ns","240μs, 649ns"
3,Keras on Tensorflow (CPU),Dense,0.418901,"1m, 12s, 988ms, 526μs, 625ns","1ms, 610μs, 799ns","38s, 130ms, 826μs, 237ns","841μs, 517ns"


In [6]:
print(results.to_html())

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Library</th>
      <th>Model</th>
      <th>LogLoss</th>
      <th>Fit time</th>
      <th>Average fit time</th>
      <th>Predict time</th>
      <th>Average predict time</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>creme</td>
      <td>LogisticRegression</td>
      <td>0.413533</td>
      <td>812ms, 676μs, 174ns</td>
      <td>17μs, 935ns</td>
      <td>245ms, 461μs, 635ns</td>
      <td>5μs, 417ns</td>
    </tr>
    <tr>
      <th>1</th>
      <td>scikit-learn</td>
      <td>SGDClassifier</td>
      <td>0.413533</td>
      <td>18s, 902ms, 205μs, 239ns</td>
      <td>417μs, 157ns</td>
      <td>6s, 652ms, 347μs, 9ns</td>
      <td>146μs, 812ns</td>
    </tr>
    <tr>
      <th>2</th>
      <td>PyTorch (CPU)</td>
      <td>Linear</td>
      <td>0.418901</td>
      <td>30s, 274ms, 892μs, 26ns</td>
      <td>668μs, 143ns</td>
      <td>10s, 904ms, 302μs